<a href="https://colab.research.google.com/github/heefan/deeplearning_ai_courses/blob/main/ai_agents_in_langraph/ex1/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>